<div style="max-width:1400px;margin-center: auto">
<img src="images\race join.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("race_join") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [2]:
races_df = spark.read.csv(r'E:\unused\Udemy\Spark_practice\raw\raw_files\races.csv', header =True, inferSchema =True).\
            withColumnRenamed('year','race_year').\
            withColumnRenamed('name','race_name').\
            withColumnRenamed('date','race_date')
constructors_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\constructors').\
            withColumnRenamed('name','team')
drivers_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\drivers').\
            withColumnRenamed('name','driver_name').\
            withColumnRenamed('number','driver_number').\
            withColumnRenamed('nationality','driver_nationality')
circuit_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\circuit').\
                    withColumnRenamed('location','circuit_location')
results_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\processed\results').\
            withColumnRenamed('time','race_time')
races_df.show(2)
constructors_df.show(2)
drivers_df.show(2)
circuit_df.show(2)
results_df.show(2)

+------+---------+-----+---------+--------------------+-------------------+--------+--------------------+
|raceId|race_year|round|circuitId|           race_name|          race_date|    time|                 url|
+------+---------+-----+---------+--------------------+-------------------+--------+--------------------+
|     1|     2009|    1|        1|Australian Grand ...|2009-03-29 00:00:00|06:00:00|http://en.wikiped...|
|     2|     2009|    2|        2|Malaysian Grand Prix|2009-04-05 00:00:00|09:00:00|http://en.wikiped...|
+------+---------+-----+---------+--------------------+-------------------+--------+--------------------+
only showing top 2 rows

+--------------+---------------+----------+-----------+--------------------+
|constructor_id|constructor_ref|      team|nationality|      ingestion_date|
+--------------+---------------+----------+-----------+--------------------+
|             1|           null|   McLaren|    British|2023-12-31 16:08:...|
|             2|           null

In [3]:
fnl_df = results_df.join(constructors_df, results_df.constructor_id == constructors_df.constructor_id,'inner').\
        join(drivers_df,results_df.driver_id == drivers_df.driver_id,'inner').\
        join(races_df,results_df.race_id == races_df.raceId,'inner').\
        join(circuit_df,races_df.circuitId == circuit_df.circuit_id,'inner').\
        select(races_df.race_year,
               races_df.race_name,
               races_df.race_date,
               circuit_df.circuit_location,
               drivers_df.driver_name,
               drivers_df.driver_number,
               drivers_df.driver_nationality,
               constructors_df.team,
               results_df.grid,
               results_df.fastest_lap,
               results_df.race_time,
               results_df.points,
              results_df.position)
fnl_df.show(2)

+---------+----------------+-------------------+----------------+-------------+-------------+------------------+------------+----+-----------+----------+------+--------+
|race_year|       race_name|          race_date|circuit_location|  driver_name|driver_number|driver_nationality|        team|grid|fastest_lap| race_time|points|position|
+---------+----------------+-------------------+----------------+-------------+-------------+------------------+------------+----+-----------+----------+------+--------+
|     1954|Indianapolis 500|1954-05-31 00:00:00|    Indianapolis|Bill Vukovich|         null|          American|Kurtis Kraft|  19|       null|3:49:17.27|   8.0|       1|
|     1954|Indianapolis 500|1954-05-31 00:00:00|    Indianapolis|  Jimmy Bryan|         null|          American|       Kuzma|   3|       null|  +1:09.95|   6.0|       2|
+---------+----------------+-------------------+----------------+-------------+-------------+------------------+------------+----+-----------+--------

In [4]:
filtered = fnl_df.filter("race_year == 2020 and race_name == 'Abu Dhabi Grand Prix'").\
                orderBy(fnl_df.points.desc())
filtered.show(5)

+---------+--------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+
|race_year|           race_name|          race_date|circuit_location|    driver_name|driver_number|driver_nationality|    team|grid|fastest_lap|  race_time|points|position|
+---------+--------------------+-------------------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+
|     2020|Abu Dhabi Grand Prix|2020-12-13 00:00:00|       Abu Dhabi| Max Verstappen|           33|             Dutch|Red Bull|   1|         14|1:36:28.645|  25.0|       1|
|     2020|Abu Dhabi Grand Prix|2020-12-13 00:00:00|       Abu Dhabi|Valtteri Bottas|           77|           Finnish|Mercedes|   2|         40|    +15.976|  18.0|       2|
|     2020|Abu Dhabi Grand Prix|2020-12-13 00:00:00|       Abu Dhabi| Lewis Hamilton|           44|           British|Mercedes|   3|   

In [5]:
fnl_df.write.format('parquet').saveAsTable('f1_presentation.race_result')